# Limpieza de los datos

In [18]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
from collections import Counter
import re
import nltk
nltk.download(['punkt','stopwords','wordnet','words'])
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
#import unidecode

[nltk_data] Downloading package punkt to /home/cmejia3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cmejia3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/cmejia3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /home/cmejia3/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [19]:
from nltk import *
from nltk.corpus import *

In [20]:
base_path =  Path("~").expanduser().resolve()
#base_path = Path.cwd().expanduser().resolve()
input_file_path  = base_path / 'datasets/papers-txt/'
#datasetOut =base_path / "datasets/salidas_procesamiento/"
#datasetOut_freq = base_path / "datasets/salidas_freq/"

In [21]:
input_file_path

PosixPath('/home/cmejia3/datasets/papers-txt')

# Preprocesamiento

In [22]:
matcher= re.compile(r'(.)\1*')

In [23]:
matcher= re.compile(r'(.)\1*')
tot = [len(match.group()) for match in matcher.finditer('aaaadaaacbbabqq')]

In [24]:
max(tot)

4

In [25]:
def fileSize(fileIn):
    size=os.stat(fileIn).st_size
    return size

def wordsCount(contenido):
    totWwords=contenido.split()
    return len(totWwords)

def cleanWordCount(contenido):
    contenido =re.sub('(f|ht)tp(s?)://(.*)[.][a-z]+',' ',contenido) # Eliminar las URL
    #contenido =re.sub('REFERENCES (\S|\w)+',' ',contenido) # Eliminar la bibliografia
    contenido =re.sub('[a-zA-Z0-9.?{}]+@\w+\.\w+.\w*','',contenido) # Eliminar los correos
    contenido =re.sub('\[[a-zA-Z0-9\,\. ]+\]','',contenido) # Eliminar cualquier contenido entre corchetes
    contenido =re.sub('\([a-zA-Z0-9\,\.\- ]+\)',' ',contenido) # Eliminar cualquier contenido entre paréntesis
    contenido =re.sub('((et al\.)|(i\.i\.d\.)|(i\.e\.)|\-|\'|\’|\`)','',contenido) # Eliminar abreviaciones, apostrofes y guion
    #contenido =re.sub('(f|F)igure [0-9]+.[0-9]',' ',contenido) # Eliminar Figure
    contenido =re.sub('[^a-zA-Z_á\éíóúà\èìòùäëïöü\s]','',contenido) # Eliminar caracteres que no sean: letra, número o vocales acentuadas
    contenido =re.sub(' +',' ',contenido) # Eliminar espacios en blanco
    contenido =re.sub('(a-z|A-Z){1,1}','',contenido) # Eliminar palabras o números de un caracter de longitud   
    #contenido =re.sub('[^A-Za-z0-9.,_%+-\(\)\[\]\´\'\`]',' ',contenido)
    #contenido =re.sub('\[(0-9)+\]',' ',contenido)    
    #totWordDepurado = Counter(map(str, contenido.split()))
    #outputFile= open(datasetOut, 'w', encoding='UTF-8')
    #outputFile.write(contenido)
    #outputFile.close()
    totWwords=contenido.split()
    setWords = set(totWwords)
    #print("Total de palabras {}".format(len(totWwords)))
    #print("Total de palabras después del pre-procesamiento: {}".format(totWordDepurado))
    return len(totWwords),contenido,setWords

# Procesamiento

In [26]:
def lang_ratio(input):
    lang_ratio = {}
    tokens = wordpunct_tokenize(input)
    words = [word.lower() for word in tokens]
    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        word_set = set(words)
        common_elements = word_set.intersection(stopwords_set)
        lang_ratio[language] = len(common_elements)
    return lang_ratio

def detect_language(input):
    ratios = lang_ratio(input)
    language = max(ratios, key = ratios.get)
    return language

Conjunto de las stop words que serán eliminadas ya que no aportan valor:

In [27]:
stopWords = stopwords.words('english')

Palabras que considermos StopWords que no estan incluidas en el conjunto descargado:

In [28]:
listoStopWords = ['www','https','html','figure', 'chapter','abbcbccabcabcabcabcbcbabacba']

Se añaden las palabras que consideramos al conjunto principal

In [29]:
stopWords.extend(listoStopWords)

In [30]:
dictonary = nltk.corpus.words.words()

### Función principal procesamiento

In [31]:
def clean_files(texto,stopWords):
    
    #Quitar todos los acentos
    #texto = unidecode.unidecode(texto)
    
    #Quitar todos los caracteres especiales
    texto = re.sub('[^A-Za-z0-9]+',' ',texto)
    
    #Pasar todo a minisculas
    texto = texto.lower()
    
    #Tokenizar
    tokens = texto.split()
    
    #Variable que guarda el año en el que estamos que es el limite superior de los números que no se van a eliminar
    currentYear = int(dt.datetime.now().year)
    
    #Verificar que las palabras tengan más de un caracter, que además sean solo sean letras
    # o si son números que esten entre un rango que sea admisible para no eliminar información de año que se mencione en los artículos
    # y finalmente que no sean palabras que estan en el dicccionario de stopwords.
    
    #tokens = [w for w in tokens if (len(w)>1)&(w.isalpha() or (w.isnumeric() and int(w)>=1800 and int(w)<=currentYear))&(w not in stopWords)]
    tokens = [w for w in tokens if (len(w)>1)&(w.isalpha())&(w not in stopWords)]
    
    matcher= re.compile(r'(.)\1*')
    tokens1 = []
    for word in tokens:
        aux = [len(match.group()) for match in matcher.finditer(word)]
        if max(aux)<=3:
            tokens1.append(word)
        else:
            print(word)
    
    
    tokens1 = [w for w in tokens1 if (Counter(w).most_common(1)[0][1]<5)]
    eliminadas = [w for w in tokens1 if (Counter(w).most_common(1)[0][1]>=5)]
    #Stemmer
    ps = PorterStemmer() 
    tokens1 = [ps.stem(w) for w in tokens1]
    
    #Lematización
    word_net_lemmatizar = WordNetLemmatizer()

    tokens1 = [word_net_lemmatizar.lemmatize(w, pos = "v") for w in tokens1]
    
    #Se retorna el texto nuevamente en un solo string luego de ser procesado
    to_return = ' '.join(tokens1)
    
    #Se retorna el vocabulario de cada documento
    set_words = set(tokens1)
    
    #Y la frecuencia de las palabras
    freq = nltk.FreqDist(tokens1)
    return to_return,set_words,freq,eliminadas

Inicialización de los conjuntos:

    - Vocabulary: el conjunto de todas las palabras que contienen los documentos
    - results_text: la lista con los documentos ya organizados para construir el bag of words
    - results_frecuency: información de cada documento de las palabras que contiene cuántas veces las contiene

In [32]:
vocabulary = set()
results_text = []
results_frecuency = []

## Limpieza de los datos

In [33]:
#fileSummary = "CleanSummary_copy.csv"

contenido= "Archivo" + ";" + "Tamaño(K)" + ";" + "Cant Palabras Inicial" + ";" + "Cant Palabras depuradas"+ ";" +"Porc Limpieza"+ "\n"
resultado = pd.DataFrame()
indexFiles = []
documents = []
i = 0
for f in input_file_path.glob('*.txt'):
    #Peso del archivo
    tmpSize=round(fileSize(f)/1014)
    
    #Lectura del archivo
    input_file = open(f, "r", encoding = 'utf-8')
    input_aux = input_file.read()
    
    #Cuenta de palabras iniciales
    tmpWordsOri=wordsCount(input_aux)
    #out = datasetOut / str(f).split('/')[-1]
    
    #Cuenta de palabras despues de la limpieza
    tmpWordsEnd,text,setWord=cleanWordCount(input_aux)
    #tmpPerClean=round((tmpWordsEnd/tmpWordsOri)*100)
    
    
    #Detectando el idioma
    aux = detect_language(text)
    
    if(aux == 'english'):
        i = i+1
        text_cleanned,set_words,freq,eliminadas = clean_files(text,stopWords)
        if len(eliminadas)> 0:
            print(eliminadas)
        #out2 = datasetOut / str(f).split('/')[-1]
        documents.append(text_cleanned)
        vocabulary = vocabulary.union(set_words)
        results_text.append(text_cleanned)
        indexFiles.append(str(f).split('/')[-1])
        
        #Escritura de los resultados del preprocesamiento
        auxRes= pd.DataFrame({'Archivo': str(f).split('/')[-1], 'Tamaño(K)': [tmpSize], 'Cant Palabras Inicial': [tmpWordsOri],'Cant Palabras depuradas': [tmpWordsEnd],"Vocabulario Inicial":len(setWord),"Vocabulario Final":len(set_words)})
        resultado = pd.concat([resultado, auxRes])
    else:
        print(aux + ':' + str(f))
    break
#resultado.to_csv(fileSummary, sep = ';', index = False)

In [34]:
print(vocabulary)

{'disequilibrium', 'kartik', 'homogen', 'collect', 'snitkin', 'tool', 'actuat', 'email', 'increas', 'experiment', 'sourc', 'aris', 'style', 'absorb', 'collett', 'carri', 'comp', 'model', 'passerini', 'freez', 'span', 'distinguish', 'four', 'appli', 'physic', 'verifi', 'spatial', 'shakhnovich', 'thermodynamischen', 'reduct', 'toward', 'quantum', 'lower', 'occur', 'physik', 'larger', 'within', 'xand', 'intelligent', 'microscop', 'shanghai', 'paper', 'arrow', 'simultan', 'purpos', 'unreach', 'zhu', 'object', 'without', 'human', 'gate', 'least', 'cube', 'relat', 'macroscop', 'grain', 'discuss', 'entropieverminderung', 'gc', 'respons', 'rna', 'indic', 'nandnor', 'superfamili', 'solid', 'motif', 'chromosom', 'kb', 'jiao', 'hu', 'quantiti', 'otherwis', 'percept', 'form', 'classif', 'enough', 'muld', 'design', 'becom', 'rd', 'spontan', 'trainlik', 'verif', 'delta', 'could', 'ri', 'twolink', 'gain', 'alon', 'doinatur', 'basic', 'ginestra', 'memori', 'suffici', 'state', 'stabl', 'element', 'vali

In [35]:
input_aux

'Entropy NAND: Early Functional Completeness in Entropy Networks\nForrest Fabian Jesse1,2*\n1\n\nSchool of Computer and Information Technology, Beijing Jiao Tong University, Beijing,\n100044, China\n\n2\n\nBio-X Institutes, Shanghai Jiao Tong University, Shanghai, 200240, China\n\n* Author to whom correspondence should be addressed; email: jesse@jesse.org;\n\nAbstract: An observer increases in relative entropy as it receives information from what it is\nobserving. In a system of only an observer and the observed, an increase in the relative\nentropy of the observer is a decrease in the relative entropy of the observed. Linking together\nthese directional entropy disequilibriums we show that NAND and NOR functionality arise\nin such networks at very low levels of complexity.\nKeywords: observation; networks; entropy transfer; directed entropy; directed observation;\nfunctionally complete; NAND, NOR\n\n1. Introduction\nWe describe a simple structure of directed entropy transfer which we 

In [36]:
set_words

{'absolut',
 'absorb',
 'absorpt',
 'abstract',
 'abstractli',
 'accardi',
 'accord',
 'account',
 'acid',
 'acquir',
 'across',
 'actuat',
 'addit',
 'address',
 'adler',
 'allow',
 'almost',
 'alon',
 'alreadi',
 'also',
 'anand',
 'annal',
 'anoth',
 'antoin',
 'appear',
 'appli',
 'applic',
 'appropri',
 'arakelyan',
 'architectur',
 'area',
 'aris',
 'arrang',
 'arrow',
 'artak',
 'articl',
 'artyom',
 'artzyrandrup',
 'arxiv',
 'arxivnlin',
 'ashenberg',
 'associ',
 'attain',
 'author',
 'avignon',
 'back',
 'bacteri',
 'base',
 'basi',
 'basic',
 'becom',
 'begin',
 'bei',
 'beij',
 'bela',
 'ber',
 'berlin',
 'berut',
 'bianconi',
 'binari',
 'bioinformat',
 'biol',
 'biolog',
 'biox',
 'bistat',
 'bite',
 'block',
 'boltzmann',
 'bottom',
 'brian',
 'brief',
 'brillouin',
 'build',
 'capac',
 'caret',
 'carri',
 'case',
 'central',
 'chang',
 'character',
 'charl',
 'china',
 'chklovskii',
 'chromosom',
 'ciliberto',
 'circl',
 'circuit',
 'class',
 'classif',
 'classifi',
 'c

In [37]:
resultado.head()

,Archivo,Tamaño(K),Cant Palabras Inicial,Cant Palabras depuradas,Vocabulario Inicial,Vocabulario Final
0,1410.2670.txt,24,3926,3487,939,608


In [38]:
len(vocabulary)

608

## Construcción del Bag of Words

In [39]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import vstack,save_npz,load_npz

Construido el vocabulario podemos construir el bag of words, que se hace con la ayuda de la funcion CountVectorizer

In [40]:
vectorizer = CountVectorizer(analyzer = "word",vocabulary =vocabulary , tokenizer = None, preprocessor = None, stop_words = 'english', max_features = 5000) 
train_data_features = vectorizer.fit_transform(documents)

In [41]:
len(vocabulary)

608

In [42]:
save_npz('sparse_matrix.npz', train_data_features)
#sparse_matrix = load_npz('sparse_matrix.npz')

In [43]:
with open('vocabulary.txt', 'w') as f:
    for item in vectorizer.get_feature_names():
        f.write("%s\n" % item)

In [44]:
import pickle
filename = 'BoW1.sav'
pickle.dump(vectorizer, open(filename, 'wb'))
#loaded_model = pickle.load(open('BoW1.sav', 'rb'))

In [45]:
#words = set(nltk.corpus.words.words())

In [46]:
#len(vocabulary)

In [47]:
#print(vocabulary - words)